Preliminary analysis using causal inference on single cell data

In [56]:
import pandas as pd
import numpy
import scanpy
from collections import Counter

In [21]:
ann_data = scanpy.read_h5ad('preprocessed_adata/Weinreb2020.adata.h5ad')

In [24]:
ann_data

AnnData object with n_obs × n_vars = 130887 × 2447
    obs: 'Time point', 'Population', 'Annotation', 'Well', 'cell_idx', 'clone_idx'
    var: 'gene_id', 'highly_variable'
    uns: 'highly_variable_genes_idx'
    obsm: 'X_clone', 'X_pca', 'X_spring', 'X_umap'
    layers: 'X_scaled'

In [31]:
ann_data.layers['X_scaled']

array([[-0.01457937, -0.01723505, -0.01802866, ..., -0.01229994,
        -0.02931115, -0.01147309],
       [-0.01457937, -0.01723505, -0.01802866, ..., -0.01229994,
        -0.02931115, -0.01147309],
       [-0.01457937, -0.01723505, -0.01802866, ..., -0.01229994,
        -0.02931115, -0.01147309],
       ...,
       [-0.01457937, -0.01723505, -0.01802866, ..., -0.01229994,
        -0.02931115, -0.01147309],
       [-0.01457937, -0.01723505, -0.01802866, ..., -0.01229994,
        -0.02931115, -0.01147309],
       [-0.01457937, -0.01723505, -0.01802866, ..., -0.01229994,
        -0.02931115, -0.01147309]], dtype=float32)

In [41]:
ann_data.to_df().head()

,24,41,60,71,77,88,99,148,173,177,...,24398,24416,24499,24504,24506,24507,24520,24525,24530,24536
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.861461,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.481799,0.0,0.481799,0.0,4.336191,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.403299,0.0,0.000000,0.0,4.032991,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.632407,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [59]:
c = Counter(ann_data.obs['clone_idx'].dropna().tolist())

In [71]:
top10_clones = [clone[0] for clone in c.most_common(10)]

In [72]:
top10_clones

[1261.0, 2370.0, 5714.0, 292.0, 5209.0, 3345.0, 1843.0, 2256.0, 5150.0, 2673.0]

In [74]:
# task: get the cells in the three most common clones
top10_data = ann_data[ann_data.obs['clone_idx'].isin(top10_clones)].copy()

In [76]:
top10_data

AnnData object with n_obs × n_vars = 1316 × 2447
    obs: 'Time point', 'Population', 'Annotation', 'Well', 'cell_idx', 'clone_idx'
    var: 'gene_id', 'highly_variable'
    uns: 'highly_variable_genes_idx'
    obsm: 'X_clone', 'X_pca', 'X_spring', 'X_umap'
    layers: 'X_scaled'

In [155]:
mouse_TFs = pd.read_csv('Mus_musculus_TF.txt', sep='\t')

In [87]:
mouse_TF_data = top10_data[:, top10_data.var['gene_id'].isin(mouse_TFs['Symbol'].tolist())]

In [78]:
# need to read a list of transcription factors--maybe the list they used in their analysis. Unfortunately that link is broken, so we'll use a different list: https://www.nature.com/articles/ncomms15089#Abs1

In [89]:
data = mouse_TF_data

Step 1: find clones that differentially express a transcription factor--I hope this works!

I would make a panel of scatterplots, one scatterplot for each clone. But there are too many transcription factors. Filter for really nicely known ones? Start with just 1!

In [117]:
data = ann_data[ann_data.obs['clone_idx'].isin(top100_clones)].copy()

In [123]:
data

AnnData object with n_obs × n_vars = 7381 × 2447
    obs: 'Time point', 'Population', 'Annotation', 'Well', 'cell_idx', 'clone_idx'
    var: 'gene_id', 'highly_variable'
    uns: 'highly_variable_genes_idx'
    obsm: 'X_clone', 'X_pca', 'X_spring', 'X_umap'
    layers: 'X_scaled'

In [144]:
single_clone.X.max()

0.0

In [149]:
for clone in top100_clones:
    single_clone = data[data.obs['clone_idx'] == clone, data.var['gene_id'] == 'Trp63']
    if single_clone.X.max() > 0:
        print(single_clone.to_df().iloc[:, 0].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5328056812286377, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.614198625087738, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

I'm inclined to do Myc, Sox, or Trp. Zoom in on Trp63

Trp63 targets https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2413161/ 

In [107]:
# maybe top ten clones isn't going to work? 

In [110]:
top10_data.X.mean()

0.39272532

In [150]:
# maybe query the data frame that has 100 clones and 160 TFs

In [156]:
data = ann_data[ann_data.obs.clone_idx.isin(top100_clones), ann_data.var.gene_id.isin(mouse_TFs['Symbol'].tolist())]

In [157]:
data

View of AnnData object with n_obs × n_vars = 7381 × 161
    obs: 'Time point', 'Population', 'Annotation', 'Well', 'cell_idx', 'clone_idx'
    var: 'gene_id', 'highly_variable'
    uns: 'highly_variable_genes_idx'
    obsm: 'X_clone', 'X_pca', 'X_spring', 'X_umap'
    layers: 'X_scaled'

In [160]:
df = data.to_df()

In [167]:
(df.astype(bool).sum() > 100).tolist()

[True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True

In [171]:
df.loc[:, (df.astype(bool).sum() > 100).tolist()]

,2374,3087,3089,3512,3528,3574,4583,4584,4585,5328,...,21391,21746,21747,22242,22245,22321,23018,23489,23505,23605
2,0.0,0.0,0.00000,0.000000,0.481799,0.000000,3.372593,0.481799,0.000000,0.0,...,0.481799,0.000000,0.0,0.000000,0.481799,0.0,0.000000,0.000000,0.481799,2.408995
15,0.0,0.0,0.00000,0.377392,0.000000,0.000000,1.132177,1.132177,0.377392,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.886962
16,0.0,0.0,0.00000,0.000000,0.000000,0.000000,2.259615,1.694711,0.000000,0.0,...,1.694711,0.564904,0.0,1.129807,0.000000,0.0,0.564904,0.564904,1.129807,0.564904
25,0.0,0.0,0.00000,0.356325,0.000000,0.000000,1.068975,0.356325,0.356325,0.0,...,0.178162,0.356325,0.0,0.000000,0.178162,0.0,0.000000,0.000000,0.000000,1.425300
30,0.0,0.0,0.84396,0.000000,0.000000,0.000000,3.375842,0.843960,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.843960,0.0,0.000000,0.000000,0.000000,0.843960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128665,0.0,0.0,0.00000,0.000000,1.512743,0.000000,4.538228,0.000000,1.512743,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.512743,0.000000,0.000000,1.512743
129010,0.0,0.0,0.00000,0.000000,0.000000,0.984861,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.939444
129012,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,1.686438,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
130495,0.0,0.0,0.00000,0.000000,0.000000,3.886700,0.000000,1.295567,2.591133,0.0,...,0.000000,0.000000,0.0,1.295567,0.000000,0.0,0.000000,1.295567,0.000000,0.000000


In [159]:
# now we want to systematically query differential expression of these T

2374         Ahr
2523        Alx4
2907      Arid3c
2960       Arnt2
2996         Arx
          ...   
23489    Tsc22d1
23505      Tshz1
23507      Tshz3
23605       Ttf1
24050       Vax2
Name: gene_id, Length: 161, dtype: object